In [1]:
from pathlib import Path
import os
import napari
import nd2
import numpy as np
import tifffile
import pandas as pd
from skimage import measure
from skimage.transform import resize
from scipy.ndimage import binary_fill_holes
from utils import get_gpu_details, list_images, read_image

get_gpu_details()

Device name: /device:GPU:0
Device type: GPU
GPU model: device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("./raw_data/nihanseb_organoid")

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
# Now, in addition to xy, you can downsample across your z-stack
slicing_factor_xy = 4 # Use 2 or 4 for downsampling in xy (None for lossless)
slicing_factor_z = None # Use 2 to select 1 out of every 2 z-slices

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 2

# Fill holes inside the resulting organoid mask? Set to False if you want to keep the holes
fill_holes = True

# Analyze intensity within the 3D volume of the ROI, or perform a mean or max intensity projection of the marker channel (2D)
analysis_type = "2D" #"2D" or "3D"

# If 2D analysis type, Choose projection type (mean intensity or max intensity)
# Mean intensity projection would be the equivalent of analyzing avg_intensity within the 3D volume
projection_type = "mean" # "mean" or "max"

# Stardist model name if nuclei labels predictions are present
model_name = None

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['raw_data\\nihanseb_organoid\\MLD 1.8 block4 ARSA MBP batch 1 40x.nd2',
 'raw_data\\nihanseb_organoid\\MLD 2.2 block7 MBP MAP2 slide 7 batch 2 40x.nd2']

In [11]:
# Explore each image to analyze (0 defines the first image in the directory)
image = images[0]

# Read image, apply slicing if needed and return filename and img as a np array
img, filename = read_image(image, slicing_factor_xy, slicing_factor_z)

# Generate maximum or mean intensity projection
if projection_type == "max":
    img_projection = np.max(img, axis=1)
elif projection_type == "mean":
    img_projection = np.mean(img, axis=1)

# Show image in Napari
viewer = napari.Viewer(ndisplay=2)
viewer.add_image(img_projection, name=f"{projection_type}_projection")

# Add the 3D-stack into Napari
if analysis_type == "3D":
    # Remove the projection layer if it exists
    if f"{projection_type}_projection" in viewer.layers:
        viewer.layers.remove(f"{projection_type}_projection")
    # Add the 'img' stack
    viewer.add_image(img)
    # Set projection_type variable to None
    projection_type = None



Image analyzed: MLD 1.8 block4 ARSA MBP batch 1 40x
Original Array shape: (3, 24, 10797, 10797)
Compressed Array shape: (3, 24, 2700, 2700)


In [12]:
# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, min_max_range),(..., ...)]
# Remember in Python one starts counting from 0, so your first channel will be 0
# min_max range defines the pixel intensity range within which a cell is considered positive for a marker
# i.e. markers = [("ARSA", 0, (0, 65536)), ("MBP", 1, (0, 65536))]
markers = [("ARSA", 0, (110, 1000)), ("MBP", 1, (105, 200))]

In [20]:
# Construct ROI and nuclei predictions paths from directory_path above
roi_path = directory_path / "ROIs"
# nuclei_preds_path =  directory_path / "nuclei_preds" / analysis_type / model_name

# Extract the experiment name from the data directory path
experiment_id = directory_path.name

# Check for presence of ROIs
try:
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]

except FileNotFoundError:
    roi_names = ["auto_generated_ROI"]
    print("No manually defined ROI found, generating ROI automatically...")

# Create a 'results' folder in the root directory to store results
results_folder = Path("results") / experiment_id

try:
    os.makedirs(results_folder)
    print(f"'{results_folder}' folder created successfully.")
except FileExistsError:
    print(f"'{results_folder}' folder already exists.")

for roi_name in roi_names:

    print(f"\nAnalyzing ROI: {roi_name}")

    # Initialize an empty list to hold the extracted dataframes on a per channel basis
    props_list = []

    # Read the user defined ROIs, in case of missing ROI implement logic for automatic segmentation
    try:
        # Read previously defined ROIs
        organoid_mask = tifffile.imread(roi_path / roi_name / f"{filename}.tiff")

    except FileNotFoundError:
        # Add logic to automatically generate an organoid mask
        pass

    # Resample the organoid ROI if input img and ROI shape differ
    if organoid_mask.shape[-2:] != img.shape[-2:]:
        roi_slicing_factor = organoid_mask.shape[-1] / img.shape[-1]
        
        if roi_slicing_factor > 1:
            print("Slicing ROI to match input image shape")
            roi_slicing_factor = round(organoid_mask.shape[-1] / img.shape[-1])
            organoid_mask = organoid_mask[::round(roi_slicing_factor), ::round(roi_slicing_factor)]
    
        elif roi_slicing_factor < 1:
            print("Upsampling ROI to match input image shape")
            organoid_mask = resize(
                organoid_mask, img.shape[-2:], order=0, preserve_range=True, anti_aliasing=False
            )

    # If analysis type == "3D" extend ROI over the entire volume
    if analysis_type == "3D":
        # Extract the number of z-slices to extend the mask
        slice_nr = img.shape[1]
        # Extend the mask across the entire volume
        organoid_mask = np.tile(organoid_mask, (slice_nr, 1, 1))
        
    # Show organoid ROI over input image 
    viewer.add_labels(organoid_mask)

    if fill_holes:
        # Close empty holes surrounded by True pixels
        organoid_mask = binary_fill_holes(organoid_mask)
        viewer.add_labels(organoid_mask, name="closed_organoids_mask")

    # Transform organoid mask into a label type without the need to perform connected components
    organoid_mask = organoid_mask.astype(np.uint8)

    # Initialize an empty list to hold the extracted dataframes on a per channel basis
    props_list = []

    # Create a dictionary containing all image descriptors
    descriptor_dict = {
                "filename": filename,
                "roi": roi_name,
                "fill_holes": fill_holes,
                "slicing_factor_xy": slicing_factor_xy,
                "analysis_type": analysis_type,
                "projection_type": projection_type,
                }

    for channel_name, ch_nr, min_max_range in markers:

        if analysis_type == "2D":
            # Ignore pixel values below the min_range (set them to 0)
            img_projection[ch_nr] = np.where(img_projection[ch_nr] > min_max_range[0], img_projection[ch_nr], 0)
            viewer.add_image(img_projection[ch_nr], name=f"filtered_{channel_name}_input")

            # Ignore pixels whose value is equal or above the max_range
            # ROI is modified to ignore said pixels (results in filtered organoid_mask)

            filtered_organoid_mask = np.where(img_projection[ch_nr] <= min_max_range[1], organoid_mask, 0)
            viewer.add_labels(filtered_organoid_mask, name=f"filtered_roi_for_{channel_name}")

            # Extract area information from each marker channel
            props = measure.regionprops_table(label_image=filtered_organoid_mask,
                                    properties=["area"])
            
        elif analysis_type == "3D":
            # Ignore pixel values below the min_range (set them to 0)
            img[ch_nr] = np.where(img[ch_nr] > min_max_range[0], img[ch_nr], 0)
            viewer.add_image(img[ch_nr], name=f"filtered_{channel_name}_input")

            # Ignore pixels whose value is equal or above the max_range
            # ROI is modified to ignore said pixels (results in filtered organoid_mask)

            filtered_organoid_mask = np.where(img[ch_nr] <= min_max_range[1], organoid_mask, 0)
            viewer.add_labels(filtered_organoid_mask, name=f"filtered_roi_for_{channel_name}")

            # Extract area (volume) information from each marker channel
            props = measure.regionprops_table(label_image=filtered_organoid_mask,
                                    properties=["area"])
            
        print(f"Extracting % of ROI + for {channel_name} inside {analysis_type}_{roi_name}")
        
        # Convert to dataframe
        props_df = pd.DataFrame(props)

        # Rename area column to indicate the specific image
        props_df.rename(columns={"area": f"{channel_name}_area"}, inplace=True)

        # Append each props_df to props_list
        props_list.append(props_df)

    # Initialize the df with the first df in the list
    props_df = props_list[0]

    # Add each key-value pair from descriptor_dict to props_df at the specified position
    insertion_position = 0    
    for key, value in descriptor_dict.items():
        props_df.insert(insertion_position, key, value)
        insertion_position += 1  # Increment position to maintain the order of keys in descriptor_dict

    # Define the .csv path
    csv_path = results_folder / f'{filename}_per_label_perc_of_roi.csv'

    # Append to the .csv with new data points each round
    props_df.to_csv(csv_path, mode="a", index=False, header=not os.path.isfile(csv_path))

# Show the updated .csv 
csv_df = pd.read_csv(csv_path)

csv_df

'results\nihanseb_organoid' folder already exists.

Analyzing ROI: organoid_sf4
Extracting % of ROI + for ARSA inside 3D_organoid_sf4
Extracting % of ROI + for MBP inside 3D_organoid_sf4


,filename,roi,fill_holes,slicing_factor_xy,analysis_type,projection_type,area
0,MLD 1.8 block4 ARSA MBP batch 1 40x,organoid_sf4,True,4,3D,NaN,111743478.0
1,MLD 1.8 block4 ARSA MBP batch 1 40x,organoid_sf4,True,4,3D,NaN,111743478.0


In [ ]:
with nd2.ND2File(image) as nd2_data:
    # Get the first channel's volume metadata
    first_channel = nd2_data.metadata.channels[0]
    voxel_size = first_channel.volume.axesCalibration  # X, Y, Z calibration

    # Extract pixel sizes
    pixel_size_x, pixel_size_y, voxel_size_z = voxel_size

    print(f"Pixel size: {pixel_size_x:.3f} µm x {pixel_size_y:.3f} µm")
    print(f"Voxel (Z-step) size: {voxel_size_z:.3f} µm")

In [19]:
props_list[0]

,filename,roi,fill_holes,slicing_factor_xy,analysis_type,projection_type,ARSA_area
0,MLD 1.8 block4 ARSA MBP batch 1 40x,organoid_sf4,True,4,3D,None,111743478.0
